# 4.1 The Metacircular Evaluator

In [6]:
(define (eval exp env)
  (cond ((self-evaluating? exp)
         exp)
        ((variable? exp)
         (lookup-variable-value exp env))
        ((quoted? exp)
         (text-of-quotation exp))
        ;; assignment/definition recursively call eval to comput
        ;; new value associated with a variable
        ((assignment? exp)
         (eval-assignment exp env))
        ((definition? exp)
         (eval-definition exp env))
        ((if? exp)
         (eval-if exp env))
        ;; transform into applicable procedure within environment
        ((lambda? exp)
         (eval-lambda exp env))
        ;; evaluate sequence of expressions in order
        ((begin? exp)
         (eval-begin exp env))
        ;; case analysis -> nest of if expressions which are then evaluated
        ((cond? exp)
         (eval-cond exp env))
        ((application? exp)
         (apply (eval (operator exp) env)
                (list-of-values
                 (operands exp)
                 env)))
        ((let? exp)
         (eval-let exp env))
        ((for? exp) ;; ex 4.9
         (eval-for exp env))
        ((while? exp) ;; ex 4.9
         (eval-while exp env))
        (else
         (error "Unkown expression
                type: EVAL" exp))))

;; allow access to Scheme version of apply under a different name
(define apply-in-underlying-scheme apply)

(define (apply procedure arguments)
  (cond ((primitive-procedure? procedure)
         (apply-primitive-procedure
          procedure
          arguments))
        ;; sequentially eval each expression in procedure body
        ;; extend base environment to include new frame binding
        ;; params of procedure to the arguments it is applied to
        ((compound-procedure? procedure)
         (eval-sequence
          (procedure-body procedure)
          (extend-environment
           (procedure-parameters
            procedure)
           arguments
           (procedure-environment
            procedure))))
        (else
         (error "Unkown procedure
                type: APPLY"
                procedure))))

;; produce list of arguments to which a procedure is to be applied
;; evaluate each operand -> return list of results
(define (list-of-values exps env)
  (if (no-operands? exps)
      '()
      (cons (eval (first-operand exps) env)
            (list-of-values
             (rest-operands exps)
             env))))

(define (eval-if exp env)
  (if (true? (eval (if-predicate exp) env))
      (eval (if-consequent exp) env)
      (eval (if-alternative exp) env)))

;; eval a sequence of expressions in order
;; return value of final expression
(define (eval-sequence exps env)
  (cond ((last-exp? exps)
         (eval (first-exp exps) env))
        (else
         (eval (first-exp exps) env)
         (eval-sequence (rest-exps exps)
                        env))))

;; assign value to variable
(define (eval-assignment exp env)
  (set-variable-value!
   (assignment-variable exp)
   (eval (assignment-value exp) env)
   env)
  'ok)

;; define a variable
(define (eval-definition exp env)
  (define-variable!
   (definition-variable exp)
   (eval (definition-value exp) env)
   env)
  'ok)

(define (eval-begin exp env)
  (eval-sequence
   (begin-actions exp)
   env))

(define (eval-cond exp env)
  (eval (cond->if exp) env))

(define (eval-lambda exp env)
  (make-procedure
   (lambda-parameters exp)
   (lambda-body exp)
   env))


;; expressions

(define (self-evaluating? exp)
  (cond ((number? exp) #t)
        ((string? exp) #t)
        (else #f)))

(define (variable? exp) (symbol? exp))

;; quotations: (quote <text-of-quotation>)
(define (quoted? exp)
  (tagged-list? exp 'quote))
(define (text-of-quotation exp)
  (cadr exp))

(define (tagged-list? exp tag)
  (if (pair? exp)
      (eq? (car exp) tag)
      #f))

;; assignments: (set! <var> <value>)
(define (assignment? exp)
  (tagged-list? exp 'set!))
(define (assignment-variable exp)
  (cadr exp))
(define (assignment-value exp) (caddr exp))

;; definitions: (define <var> <value>)
;; OR (define (<var> <param-1> ... <param-n>) <body>)
(define (definition? exp)
  (tagged-list? exp 'define))
(define (definition-variable exp)
  (if (symbol? (cadr exp))
      (cadr exp)
      (caadr exp)))
(define (definition-value exp)
  (if (symbol? (cadr exp))
      (caddr exp)
      (make-lambda
       (cdadr exp) ; formal parameters
       (cddr exp)))) ; body

;; (make-definition 'my-val '() 1)
;; -> (define myval 1)
;; (make-definition 'my-func` '(x y) (display (* x y))
;; -> (define (my-func x y) (display (*x y)))
(define (make-definition identifier params body)
  (if (null? params)
      (list 'define identifier body)
      (list 'define (cons identifier params) body)))

;; lambda expressions: (lambda (<param-1> ... <param-n>) <body>)
(define (lambda? exp)
  (tagged-list? exp 'lambda))
(define (lambda-parameters exp) (cadr exp))
(define (lambda-body exp) (cddr exp))
(define (make-lambda parameters body)
  (cons 'lambda (cons parameters body)))

;; conditionals: (if <predicate> <consequent> <alternative>?)
;; alternative is optional -> default to false
(define (if? exp) (tagged-list? exp 'if))
(define (if-predicate exp) (cadr exp))
(define (if-consequent exp) (caddr exp))
(define (if-alternative exp)
  (if (not (null? (cdddr exp)))
      (cadddr exp)
      'false)) ;; default to false if no alternative
(define (make-if predicate
                 consequent
                 alternative)
  (list 'if
        predicate
        consequent
        alternative))

;; begin: (begin <exp-1> ... <exp-n>)
;; package a sequence of expressions into a single expression
(define (begin? exp)
  (tagged-list? exp 'begin))
(define (begin-actions exp) (cdr exp))
(define (last-exp? seq) (null? (cdr seq)))
(define (first-exp seq) (car seq))
(define (rest-exps seq) (cdr seq))
(define (sequence->exp seq)
  (cond ((null? seq) seq)
        ((last-exp? seq) (first-exp seq))
        (else (make-begin seq))))
(define (make-begin seq) (cons 'begin seq))

;; procedure application: any compound expression not defined as an expression type above
;; (<operator> <operand-1> ... <operand-n>)
(define (application? exp) (pair? exp))
(define (operator exp) (car exp))
(define (operands exp) (cdr exp))
(define (no-operands? ops) (null? ops))
(define (first-operand ops) (car ops))
(define (rest-operands ops) (cdr ops))

;; cond: (cond ((<predicate-1> <result-1> ... <predicate-n> <result-1n) (else <result>))
;; implemented as nested if expressions
(define (cond? exp)
  (tagged-list? exp 'cond))
(define (cond-clauses exp) (cdr exp))
(define (cond-else-clause? clause)
  (eq? (cond-predicate clause) 'else))
(define (cond-predicate clause)
  (car clause))
(define (cond-actions clause)
  (cdr clause))
(define (cond->if exp)
  (expand-clauses (cond-clauses exp)))
(define (expand-clauses clauses)
  (if (null? clauses)
      'false ;; no else clause
      (let ((first (car clauses))
            (rest (cdr clauses)))
        (if (cond-else-clause? first)
            (if (null? rest)
                (sequence->exp
                 (cond-actions first))
                (error "ELSE clause is not last: COND->IF"
                       clauses))
            (make-if (cond-predicate first)
                     (sequence->exp
                      (cond-actions first))
                     (expand-clauses
                      rest))))))

## 4.1
Nested `let`s will force the operands to be evaluated in the desired order. The `let` must be nested as multiple assignments in a single let will use the underlying Scheme order of evaluation:
```scheme
(let ((foo exp1) ((bar exp2)))
  ...)
```
Is equivalent to:
```scheme
((lambda (foo bar)...) exp1 exp2)
```

In [7]:
(define (list-of-values-lr exps env)
  (if (no-operands? exps)
      '()
      (let ((left (eval (first-operand exps) env)))
        (let ((right (list-of-values-lr (rest-operands exps) env)))
          (cons left right)))))

(define (list-of-values-rl exps env)
  (if (no-operands? exps)
      '()
      (let ((right (list-of-values-lr (rest-operands exps) env)))
        (let ((left (eval (first-operand exps) env)))
          (cons right left)))))

# 4.1.2 Representing Expressions

## 4.2
1. Moving the clause for procedure applications to before the clause for assignments produces the following for `(define x 3)`:
```scheme
(eval '(define x 3) env)
(application? '(define x 3)) => true
(apply (eval 'define env) (list-of-values ('x 3) env))
(eval 'define env)
(variable? 'define) => true
(lookup-variable-value 'define env) ERROR
```
2. Changing procedure applcations to start with `call` requires changing the `application?` predicate and `operator`/`operand` selectors:
```scheme
(define (application? exp)
  (tagged-list? exp 'call))
(define (operator exp) (cadr exp))
(define (operand exp) (cddr exp))
```

## 4.3

In [8]:
(define (eval-data-directed exp env)
  (cond ((self-evaluating? exp)
         exp)
        ((variable? exp)
         (lookup-variable-value exp env))
        ((get (car exp))
         ((get (car exp)) exp env))
        ((application? exp)
         (apply (eval (operator exp) env)
                (list-of-values
                 (operands exp)
                 env)))
        (else
         (error "Unkown expression
                type: EVAL" exp))))

(define (install-eval-data-directed)
  (put 'quote text-of-quotation)
  (put 'set! eval-assignment) 
  (put 'define eval-definition)
  (put 'if eval-if)
  (put 'lambda eval-lambda)
  (put 'begin eval-begin)
  (put 'cond eval-cond)
  'done)

## 4.4
#### `and`
- Expressions evaluated left->right
- Any expression evaluates to false -> return false
    - Remaining expressions **not** evaluated
- All expressions evaluate to true -> return value of final expression
- No expressions -> return true

#### `or`
- Expressions evaluated left->right
- Any expression evalues to true -> return that value
    - Remaining expressions **not** evaluated
- All expressions evaluate to false -> return false
- No expressions -> return false

In [9]:
;; extend to include boolean values
(define (self-evaluating? exp)
  (or (number? exp)
      (string? exp)
      (boolean? exp)))

(define (and? exp)
  (tagged-list? exp 'and))
(define (or? exp)
  (tagged-list? exp 'or))
(define (boolean-exps exp)
  (cdr exp))

(define (eval-and exp env)
  (define (eval-exps exp)
    (cond ((last-exp? exp)
           (eval (first-exp exps) env))
          ((eval (first-exp exps) env)
           (eval-exps (rest-exps exp)))
          (else #f)))
  (eval-exps (boolean-exps exp)))

(define (eval-or exp env)
  (define (eval-exps exp)
    (cond ((last-exp? exp)
           (eval (first-exp exps) env))
          ((eval (first-exp exps) env) #t)
          (else (eval-exps (rest-exps exp)))))
  (eval-exps (boolean-exps exp)))


;; OR with derived expressions using nested if
(define (eval-and exp env)
  (eval (and->if (boolean-exps exp) env) env))
(define (eval-or exp env)
  (eval (or->if (boolean-exps exp) env) env))

(define (and->if exp env)
  (cond ((last-exp? exp)
         (eval (first-exp exp) env))
        (else (make-if
               (first-exp exp)
               (and->if (rest-exps exp) env)
               #f))))

(define (or->if exp env)
  (cond ((last-exp? exp)
         (eval (first-exp exp) env))
        (else (make-if
               (first-exp exp)
               #t
               (or->if (rest-exps exp) env)))))


## 4.5
Extend `cond` to support `(<test> => <recipient>)` syntax.

In [10]:
(define (cond-arrow-form? clause)
  (eq? (cadr clause) '=>))
(define (cond-recipient clause)
  (caddr clause))

;; ((<predicate-clauses>) (<result-clauses>)) -> (<result-clauses>)
;; ((<test>) => <recipient>) -> (<recipient> (<test>))
(define (cond-consequent clause predicate)
  (if (cond-arrow-form? clause)
      (list (cond-recipient clause) predicate)
      (sequence->exp (cond-actions clause))))

(define (expand-clauses clauses)
  (if (null? clauses)
      'false ;; no else clause
      (let ((first (car clauses))
            (rest (cdr clauses)))
        (if (cond-else-clause? first)
            (if (null? rest)
                (sequence->exp
                 (cond-actions first))
                (error "ELSE clause is not last: COND->IF"
                       clauses))
            (let ((predicate (cond-predicate first)))
              (make-if predicate
                       (cond-consequent first predicate)
                       (expand-clauses rest)))))))

## 4.6

In [11]:
(define (let? exp)
  (tagged-list? exp 'let))
(define (let-definitions exp)
  (cadr exp))
(define (let-parameters exp)
  (map car (let-definitions exp)))
(define (let-exps exp)
  (map cadr (let-definitions exp)))
(define (let-body exp)
  (cddr exp))

(define (let->combination exp env)
  (cons (make-lambda (let-parameters exp)
                     (let-body exp))
        (let-exps exp)))

(define (eval-let exp env)
  (eval (let->combination exp) env))

## 4.7
Nested `let` expressions have 2 properties which facilitate rewriting `let*`:
- Variables in each `let` are evaluated *before* the nested `let` that follows it
- Each `let` will be evaluated within the environment of the preceding `let`
    - Variables will therefore be accessible in the nested `let`

These are equivalent:
```scheme
(let* ((x 3) 
       (y (+ x 2))
       (z (+ x y 5)))
  (* x z))

(let ((x 3))
  (let ((y (+ x 2)))
    (let ((z (+ x y 5)))
      (* x z))))
```
1. `(x 3)` is evaluated and the variable `x` is assigned the value of `3` within the current environment.
2. `(y (+ x 2))` is evaluated within the same environment as step 1. As such, the value of `x` is already bound to `3` and is used to assign `y` to `5` in the current environment.
3. `(z (+ x y 5))` is evalued, since `x` and `y` are bound to values within the environment, `z` is assigned to `13` in the current environment.
4. `(* x z)` is evaluated within the same environment, returning `(* 3 13) => 39`

Having previously implemented `let`, using `(eval (let*->nested-lets exp) env)` within `eval` is sufficient to handle `let*` as the transformation in `let*->nested-lets` produces `let` expressions which `eval` is able to handle.

In [12]:
(define (let*? exp)
  (tagged-list? exp 'let*))
(define (let*-definitions exp)
  (cadr exp))
(define (let*-body exp)
  (caddr exp))

(define (let*->nested-lets exp)
  (define (make-let definitions)
    (cond ((last-exp? definitions)
           (append (list 'let (list (first-exp definitions)))
                   (let-body-exp))
           (else (list
                  'let
                  (list (first-exp definitions))
                  (make-let (rest-exps definitions)))))))
  (make-let (let*-let*-definitions)))

(define (eval-let* exp env)
  (eval (let*->nested-lets exp) env))

## 4.8
Named `let` -> procedure definition followed by 1 application of the procedure with initial values given by the `let`. Implement by creating a `begin` which contains the procedure definition, followed by it's application.

In [13]:
(define (named-let? exp)
  (and (let? exp)
       (variable? (cadr exp))))

(define (let-parameter-values exp) 
  (map cadr (let-definitions exp)))

;; exp is the named-let stripped of the let symbol
;; to allow original let selectors to be used
(define (named-let->combination exp)
  (let ((identifier (car exp)))
    (display identifier)
    (make-begin
     (list
      (make-definition identifier '()
            (make-lambda (let-parameters exp)
                         (let-body exp)))
      (cons identifier (let-parameter-values exp))))))

(define (let->combination exp env)
  (if (named-let? exp)
      (named-let->combination (cdr exp))
      (cons (make-lambda (let-parameters exp)
                         (let-body exp))
            (let-exps exp))))

## 4.9

### `for`
```scheme
(for <identifier> in <sequence>) <body>)
```
For every element in `<sequence>` execute `<body>` with the symbol `<identifier>` bound to the current element in `<sequence>`. For example:
```scheme
(for x in (list 1 2 3)
     (display x)
     (newline))
; 1
; 2
; 3
```

Can be represented as sequential applications of `<body>`, each passing the next value of `<identifier>`from `<sequence>`:
```scheme
(begin
 (define (body x) (display x) (newline))
 (body 1)
 (body 2)
 (body 3))
```

1. Create selectors for `<identifier>`, `<sequence>` and `<body>`
2. Build a `lambda` statement for `<body>` which takes `<identifier>` as a parameter
3. Build a list of applications of procedure from 2, each using a value from `<sequence>` - first need to evaluate `<sequence>` to get values
4. Build a `begin` statement containing list of applications from 3.



In [14]:
(define (for? exp)
  (tagged-list? exp 'for))
(define (for-identifier exp)
  (cadr exp))
(define (for-sequence exp)
  (cadddr exp))
(define (for-body exp)
  (cddddr exp))

(define (for->combination exp env)
  (define proc (make-lambda (for-identifier exp) (for-body exp)))
  (let ((sequence-values (eval (for-sequence exp) env)))
    (make-begin
    (map (lambda (x) (cons proc x))
         sequence-values))))

(define (eval-for exp env)
  (eval (for->combination exp env) env))

In [15]:
(for->combination '(for x in '(1 2 3) (display x)(newline)) '()) ;; dummy env

(begin ((lambda x (display x) (newline)) . 1) ((lambda x (display x) (newline)) . 2) ((lambda x (display x) (newline)) . 3))

### `while`
```scheme
(while <predicate> <body>)
```

Execute `<body>` repeatedly whilst `<predicate>` evaluates to `true`. If `<predicate>` evaluates to `false` stop iteration. For example (state mutation used for ease of example):
```scheme
(define x 1)
(while (< x 4) ((display x)(newline) (set! x (+ x 1))))
; 1
; 2
; 3
```

1. Define selectors for `<predicate>` and `<body>`.
2. Transform into a `begin` sequence of `if` statements:
    - if `<predicate>` -> execute `<body>`, evaluate `while` statement again
    - Else return `null` / empty list 

In [16]:
(define (while? exp)
  (tagged-list? exp 'while))
(define (while-predicate exp)
  (cadr exp))
(define (while-body exp)
  (caddr exp))

(define (while->combination exp)
  (make-if
   (while-predicate exp)
   (make-begin (list (while-body exp) exp))
   '()))

(define (eval-while exp env)
  (eval (while->combination exp) env))

In [17]:
(while->combination '(while (< x 4) ((display x)(newline) (set! x (+ x 1)))))

(if (< x 4) (begin ((display x) (newline) (set! x (+ x 1))) (while (< x 4) ((display x) (newline) (set! x (+ x 1))))) ())

## 4.10
Alter the Scheme syntax to use `def` instead of `define`
```scheme
(define (definition? exp)
  (tagged-list? exp 'def))
(define (make-definition))

(define (make-definition identifier params body)
  (if (null? params)
      (list 'def identifier body)
      (list 'def (cons identifier params) body)))
```
No changes to `eval` or `apply` required.

# 4.1.3 Evaluator Data Structures

In [18]:
;; anything that is not explicit false is true
(define (true? x)
  (not (eq? x false)))
(define (false? x)
  (eq? x false))

;; compound procedures: (procedure (<param-1>...<param-n>) <body> <env>)
(define (make-procedure parameters body env)
  (list 'procedure parameters body env))
(define (compound-procedure? p)
  (tagged-list? p 'procedure))
(define (procedure-paramters p)
  (cadr p))
(define (procedure-body p)
  (caddr p))
(define (procedure-environment p)
  (cadddr p))

;; operations on environments

;; environment: list of frames
(define (enclosing-environment env) (cdr env))
(define (first-frame env) (car env))
(define the-empty-environment '())

;; frame: pair of lists (<variables> <values>)
(define (make-frame variables values)
  (cons variables values))
(define (frame-variables frame)
  (car frame))
(define (frame-values frame)
  (cdr frame))
(define (add-binding-to-frame! var val frame)
  (set-car! frame (cons var (car frame)))
  (set-cdr! frame (cons val (cdr frame))))

;; extend environment: add a new frame of variables + values to an environment
(define (extend-environment vars vals base-env)
  (if (= (length vars) (length vals))
      (cons (make-frame vars vals) base-env)
      (if (< (length vars) (length vals))
          (error "Too many arguments supplied"
                 vars
                 vals)
          (error "Too few arguments supplied"
                 vars
                 vals))))

;; lookup variable in environment: start at first frame, moving up enclosing
;; environments until variable found else error
(define (lookup-variable-value var env)
  (define (env-loop env)
    (define (scan vars vals)
      (cond ((null? vars)
             (env-loop
              (enclosing-environment env)))
            ((eq? var (car vars))
             (car vals))
            (else (scan (cdr vars)
                        (cdr vals)))))
    (if (eq? env the-empty-environment)
        (error "Unbound variable" var)
        (let ((frame (first-frame env)))
          (scan (frame-variables frame)
                (frame-values frame)))))
  (env-loop env))

;; set variable to value in an environment: scan for variable -> set value
;; error if variable not found
(define (set-variable-value var val env)
  (define (env-loop env)
    (define (scan vars vals)
      (cond ((null? vars)
             (env-loop
              (enclosing-environment env)))
            ((eq? var (car vars))
             (set-car! vals val))
            (else (scan (cdr vars)
                        (cdr vals)))))
    (if (eq? env the-empty-environment)
        (error "Unbound variable: SET!" var)
        (let ((frame (first-frame env)))
          (scan (frame-variables frame)
                (frame-values frame)))))
  (env-loop env))


;; define variable: change binding if variable exists in *first* frame
;; else add new binding to first frame
(define (define-variable! var val env)
  (let ((frame (first-frame env)))
    (define (scan vars vals)
      (cond ((null? vars)
             (add-binding-to-frame!
              var val frame))
            ((eq? var (car vars))
             (set-car! vals val))
            (else (scan (cdr vars)
                        (cdr vals)))))
    (scan (frame-variables frame)
          (frame-values frame))))

## 4.11

In [36]:
;; frame: list of (<variable> <value>) pairs
(define (make-frame variables values)
  (map cons variables values))
(define (frame-variables frame)
  (map car frame))
(define (frame-values frame)
  (map cdr frame))
(define (add-binding-to-frame! var val frame)
  (set-car! frame (cons var val)))

;; lookup variable in environment: start at first frame, moving up enclosing
;; environments until variable found else error
(define (lookup-variable-value var env)
  (define (env-loop env)
    (define (scan rest-of-frame)
      (cond ((null? rest-of-frame)
             (env-loop
              (enclosing-environment env)))
            ((eq? var (caar rest-of-frame))
             (cdar rest-of-frame))
            (else (scan (cdr rest-of-frame)))))
    (if (eq? env the-empty-environment)
        (error "Unbound variable" var)
        (scan (first-frame env))))
  (env-loop env))

;; set variable to value in an environment: scan for variable -> set value
;; error if variable not found
(define (set-variable-value var val env)
  (define (env-loop env)
    (define (scan rest-of-frame)
      (cond ((null? rest-of-frame)
             (env-loop
              (enclosing-environment env)))
            ((eq? var (caar rest-of-frame))
             (set-cdr! (car rest-of-frame) val))
            (else (scan (cdr rest-of-frame)))))
    (if (eq? env the-empty-environment)
        (error "Unbound variable: SET!" var)
        (scan (first-frame env))))
  (env-loop env))


;; define variable: change binding if variable exists in *first* frame
;; else add new binding to first frame
(define (define-variable! var val env)
  (let ((frame (first-frame env)))
    (define (scan rest-of-frame)
      (cond ((null? rest-of-frame)
             (add-binding-to-frame!
              var val frame))
            ((eq? var (caar rest-of-frame))
             (set-cdr! (car frame) val))
            (else (scan (cdr rest-of-frame)))))
    (scan frame)))

## 4.12

Searching for a variable in an environment is common amongst `lookup-variable-value`, `set-variable-value!` and `define-variable`. The differences between each are how to handle when a variable *is* found within a frame and when a variable is *not* found in a frame:
- `lookup-variable-value`
    - Found -> return value of variable
    - Not found -> search enclosing environment for variable
- `set-variable-value!`
    - Found -> set the value of the variable
    - Not found -> search enclosing environment for variable
- `define-variable`
    - Found -> change binding of variable
    - Not found -> add binding in first frame

Abstract the environment searching behaviour into a procedure which includes parameters which are procedures for handling the variable found/not found cases.

I have defined these abstractions for both the original representation of frames (pair of lists) and the representation from 4.11 (list of pairs).

### Original Frame Representation: pair of lists `(<variables> <values>)`

In [41]:
(define (scan-env var env found-in-frame not-found-in-frame)
  (let ((frame (first-frame env)))
    (define (scan vars vals)
      (cond ((null? vars)
             (not-found-in-frame frame))
            ((eq? var (car vars))
             (found-in-frame vars vals))
            (else (scan (cdr vars)
                        (cdr vals)))))
  (if (eq? env the-empty-environment)
      (error "Unbound variable" var)
      (scan (frame-variables frame)
            (frame-values frame))))) 

(define (lookup-variable-value var env)
  (scan-env var env
            (lambda (vars vals) (car vals))
            (lambda (frame) (scan-env (enclosing-environment env)))))

(define (set-variable-value! var val env)
  (scan-env var env
            (lambda (vars vals) (set-car! vals val))
            (lambda (frame) (scan-env (enclosing-environment env)))))

(define (define-variable! var val env)
  (scan-env var env
            (lambda (vars vals) (set-car! vals val))
            (lambda (frame) (add-binding-to-frame! var val frame))))

### Frame Representation from 4.11: list of `(<variable> <value>)` pairs

In [39]:
(define (scan-env var env found-in-frame not-found-in-frame)
  (let ((frame (first-frame env)))
    (define (scan rest-of-frame)
      (cond ((null? rest-of-frame)
             (not-found-in-frame frame))
            ((eq? var (caar rest-of-frame))
             (found-in-frame (car rest-of-frame)))
            (else (scan (cdr rest-of-frame)))))
    (if (eq? env the-empty-environment)
        (error "Unbound variable" var)
        (scan frame))))

(define (lookup-variable-value var env)
  (scan-env var env
            (lambda (binding) (car binding))
            (lambda (frame) (scan-env (enclosing-environment env)))))

(define (set-variable-value! var val env)
  (scan-env var env
            (lambda (binding) (set-cdr! binding val))
            (lambda (frame) (scan-env (enclosing-environment env)))))

(define (define-variable! var val env)
  (scan-env var env
            (lambda (binding) (set-car! binding val))
            (lambda (frame) (add-binding-to-frame! var val frame))))


## 4.13

`make-unbound!` should only remove the binding in the current lexical scope (first frame). Removing bindings in global scope from within a nested scope could cause errors when other scopes attempt to access that binding (defeating the purpose of scoped environments).

I have only implemented for the original representation of frames and will continue to do so from now on.

In [26]:
(define (make-unbound! var env)
  (scan-env var env
            (lambda (vars vals)
              (set-car! vars '())
              (set-car! vals '())
              #t)
            #f))

# 4.1.4 Running the Evaluator as a Program

In [43]:
(define primitive-procedures
  (list (list 'car car)
        (list 'cdr cdr)
        (list 'cons cons)
        (list 'null? null?)
        (list '+ +)
        (list '- -)
        (list '* *)
        (list '/ /))) ;; other primitives can be added here

(define (primitive-procedure-names)
  (map car primitive-procedures))

(define (primitive-procedure-objects)
  (map (lambda (proc)
         (list 'primitive (cadr proc)))
       primitive-procedures))

(define (primitive-procedure? proc)
  (tagged-list? proc 'primitive))

(define (primitive-implementation proc)
  (cadr proc))

(define (apply-primitive-procedure proc args)
  (apply-in-underlying-scheme
   (primitive-implementation proc) args))

(define (setup-environment)
  (let ((initial-env
         (extend-environment
          (primitive-procedure-names)
          (primitive-procedure-objects)
          the-empty-environment)))
    ;; bind true/false for use in evaluating expressions
    (define-variable! 'true #t initial-env)
    (define-variable! 'false #f initial-env)
    initial-env))

(define the-global-environment
  (setup-environment))



(define input-prompt ";;; M-Eval input:")
(define output-prompt ";;; M-Eval value:")

(define (driver-loop)
  (prompt-for-input input-prompt)
  (let ((input (read)))
    (let ((output
           (eval input
                 the-global-environment)))
      (announce-output output-prompt)
      (user-print output)))
  (driver-loop))

(define (prompt-for-input string)
  (newline) (newline)
  (display string) (newline))

(define (announce-output string)
  (newline) (display string) (newline))

;; avoid printing environment part of a compound procedure
;; environment may be very large or contain cycles
(define (user-print object)
  (if (compound-procedure? object)
      (display
       (list 'compound-procedure
             (procedure-parameters object)
             (procedure-body object)
             '<procedure-env>))
      (display object)))

## 4.14
The system version of `map` will use the system version of `apply`, not the version defined in our evaluator. It will therefore be unable to interpret the arguments passed to it as they will not be in the underlying Scheme representation. 

For example, evaluating:
```scheme
(map + '(1 2) '(3 4))
```

`apply` will call `list-of-values` to obtain the values required for a procedure application:
```scheme
(apply (eval (operator exp) env)
       (list-of-values (operands exp) env))
```

`list-of-values` returns a list made by calling `eval` with each operand in the expression. `+` is defined as a primitive procedure in the evaluator, therefore it is returned as:
```scheme
'(primitive +)
```

Scheme's built in `map` will be unable to use this as it's evaluator will treat it as a `quote` literal and won't execute the intended primitive procedure `+`.

Eva's `map` works because it uses the `apply` in our evaluator, which will 'understand' the values passed to it.

# 4.1.5 Data as Programs

## 4.15

Given a procedure `p`, `try` will run forever if `(halts? p p)` evaluates to true.

For `(try try)` to run forever, `(halts? try try)` must evaluate to true, which means that `(try try)` cannot run forever.

Similarly, for `(try try)` to halt, `(halts? try try)` must evaluate to false, which means that `(try try)` cannot halt.

Both scenarios violate the intended behaviour of `halts?`.